# Titanic Competition
This jupyter notebook includes the training of ML model used to predict the survival of passengers aboard the titanic

In [16]:
import pandas as pd  # data processing, CSV file, I/O
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

In [17]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [46]:
def extract_features(df):
    features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]
    X = pd.get_dummies(df[features])
    X["Fare"] = df["Fare"]
    return X

In [49]:
X = extract_features(train)
y = train["Survived"]

In [29]:
rfc = RandomForestClassifier()

In [30]:
forest_params = [{
    "criterion": ["entropy", "gini", "log_loss"],
    "max_leaf_nodes": [5, 50, 500, 500],
    "n_estimators": [10, 100, 1000],
    "max_features": ["sqrt", "log2", None]
}]

In [31]:
clf = GridSearchCV(rfc, forest_params, cv=10, scoring="accuracy")

In [32]:
clf.fit(X, y)
print(clf.best_params_)

{'criterion': 'entropy', 'max_features': 'sqrt', 'max_leaf_nodes': 50, 'n_estimators': 10}


In [51]:
model = clf.best_estimator_
model.fit(X, y)

RandomForestClassifier(criterion='entropy', max_leaf_nodes=50, n_estimators=10)

In [52]:
def verify(model, X, y):
    verification = model.predict(X)
    error = np.sum(abs(verification - y)) / len(y)
    print(
        f"Percentage Error: {error:.0%}"
    )

In [53]:
verify(model, X, y)

Percentage Error: 13%


In [0]:
X_test = extract_features(test)

In [71]:
X_test.isnull().sum()

Pclass        0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Fare          1
dtype: int64

In [ ]:
# TODO: Is Fare highly correlated with another variable(s)

In [66]:
predictions = model.predict(X_test)

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [68]:
output = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})
# output.to_csv('submission_iter3.csv', index=False)

ValueError: array length 891 does not match index length 418